In [76]:
import torch
import numpy as np
import os
import json
from EduNLP.ModelZoo.rnn import ElmoLM
from EduNLP.Pretrain import train_elmo, ElmoTokenizer
from EduNLP.Vector import ElmoModel, T2V
from EduNLP.I2V import Elmo, get_pretrained_i2v

os.environ["WANDB_DISABLED"] = "true"

# 训练自己的 Elmo 模型
## 1. 数据

In [77]:
# 设置你的数据路径和输出路径
BASE_DIR = "../.."

data_dir = f"{BASE_DIR}/static/test_data"
output_dir = f"{BASE_DIR}/data/pretrain_test_models/elmo/"

In [78]:
def stem_data():
    _data = []
    data_path = os.path.join(data_dir, "standard_luna_data.json")
    with open(data_path, encoding="utf-8") as f:
        for line in f.readlines():
            _data.append(json.loads(line))
    return _data

train_items = stem_data()

## 2. 训练和评估

In [79]:
# 自定义训练参数
train_params = {
  # "emb_dim": 128,
  # "hid_dim": 256,
  # "batch_size": 4,
  # "epochs": 1,
  # "lr": 5e-3,
  # "device": None,
  
  "num_train_epochs": 1,
  "per_device_train_batch_size": 8,
  "save_steps": 50,
  "save_total_limit": 2,
  "logging_steps": 5,
  "gradient_accumulation_steps": 1,
  "learning_rate": 5e-4,
}

train_elmo(train_items, output_dir, train_params=train_params)

  0%|          | 0/1 [00:00<?, ?ba/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
d:\Python\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 25
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps

  0%|          | 0/4 [00:00<?, ?it/s]



Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to ../../data/pretrain_test_models/elmo/
Trainer.model is not a `PreTrainedModel`, only saving its state dict.


{'train_runtime': 2.616, 'train_samples_per_second': 9.557, 'train_steps_per_second': 1.529, 'train_loss': 11.407695770263672, 'epoch': 1.0}


'../../data/pretrain_test_models/elmo/'


## 3.使用模型

In [80]:
test_items = [
    {'ques_content': '有公式$\\FormFigureID{wrong1?}$和公式$\\FormFigureBase64{wrong2?}$，\
            如图$\\FigureID{088f15ea-8b7c-11eb-897e-b46bfc50aa29}$,\
            若$x,y$满足约束条件$\\SIFSep$，则$z=x+7 y$的最大值为$\\SIFBlank$'},
    {'ques_content': '如图$\\FigureID{088f15ea-8b7c-11eb-897e-b46bfc50aa29}$, \
            若$x,y$满足约束条件$\\SIFSep$，则$z=x+7 y$的最大值为$\\SIFBlank$'},
    {'ques_content': "已知集合$A=\\left\\{x \\mid x^2-3 x-4<0\\right\\}, \
            \\quad B=\\{-4,1,3,5\\}, \\quad$ 则$A \\cap B$的值为"}
]

### 3.1 直接加载令牌容器和模型

In [81]:
pretrained_model_dir = output_dir

model = ElmoLM.from_pretrained(pretrained_model_dir)
tokenizer = ElmoTokenizer.from_pretrained(pretrained_model_dir)

encodes = tokenizer(test_items, lambda x: x['ques_content'])
model(**encodes)

[EduNLP, INFO] All the weights of ElmoLM were initialized from the model checkpoint at ../../data/pretrain_test_models/elmo/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use ElmoLM for predictions without further training.


ElmoLMOutput([('pred_forward',
               tensor([[[ 0.0316,  0.0020,  0.0019,  ...,  0.0027,  0.0090,  0.0026],
                        [-0.0137,  0.0537, -0.0166,  ...,  0.0321, -0.0004, -0.0132],
                        [-0.0383,  0.0203,  0.0041,  ...,  0.0692,  0.0201, -0.0460],
                        ...,
                        [ 0.0666,  0.0501, -0.0172,  ..., -0.0606, -0.0963, -0.0174],
                        [ 0.0620,  0.0210, -0.0251,  ..., -0.0247, -0.0686,  0.0461],
                        [ 0.1763,  0.0575,  0.0478,  ...,  0.0165, -0.1325,  0.0375]],
               
                       [[ 0.0266,  0.0823,  0.0178,  ...,  0.0589,  0.0346,  0.0259],
                        [ 0.0152,  0.0167,  0.0491,  ...,  0.0322,  0.0288, -0.0143],
                        [ 0.0579, -0.0485, -0.0416,  ...,  0.0089,  0.0598,  0.0234],
                        ...,
                        [ 0.0528,  0.0004,  0.0037,  ...,  0.0575, -0.1175,  0.0571],
                        [-0.0164, 

### 3.2 使用 I2V 向量化

In [82]:
tokenizer_kwargs = {"tokenizer_config_dir": pretrained_model_dir}
i2v = Elmo('elmo', 'elmo', output_dir, tokenizer_kwargs=tokenizer_kwargs)

# 可以对单个题目进行表征
i_vec, t_vec = i2v(test_items[0], key=lambda x: x["ques_content"])
print(i_vec.shape) # == torch.Size([x])
print(t_vec.shape) # == torch.Size([x, x])

# 也可以对题目列表进行表征
i_vec, t_vec = i2v(test_items, key=lambda x: x["ques_content"])
print(i_vec.shape) # == torch.Size([len(test_items), x])
print(t_vec.shape) # == torch.Size([len(test_items), x, x]))


[EduNLP, INFO] All the weights of ElmoLM were initialized from the model checkpoint at ../../data/pretrain_test_models/elmo/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use ElmoLM for predictions without further training.


torch.Size([1, 600])
torch.Size([1, 17, 600])
torch.Size([3, 600])
torch.Size([3, 40, 600])


d:\Python\lib\site-packages\EduNLP\Vector\elmo_vec.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  (outputs.forward_output[torch.arange(len(items["seq_len"])), torch.tensor(items["seq_len"]) - 1],


### 3.3 使用 Tokenizer 和 T2V 向量化

In [83]:
# 加载之前训练的模型 tokenizer
tokenizer = ElmoTokenizer.from_pretrained(pretrained_model_dir)
encodes = tokenizer(test_items, key=lambda x: x['ques_content'])

t2v = ElmoModel(pretrained_model_dir)

i_vec = t2v(encodes)
print(i_vec.shape) # == torch.Size([len(test_items), x])
print()

i_vec = t2v.infer_vector(encodes)
t_vec = t2v.infer_tokens(encodes)
print(i_vec.shape) # == torch.Size([len(test_items), x])
print(t_vec.shape) # == torch.Size([len(test_items), x, x]))
print()

[EduNLP, INFO] All the weights of ElmoLM were initialized from the model checkpoint at ../../data/pretrain_test_models/elmo/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use ElmoLM for predictions without further training.


torch.Size([3, 600])

torch.Size([3, 600])
torch.Size([3, 40, 600])



### 3.4 使用 EduNLP 中公开的预训练模型

In [85]:
# 获取公开的预训练模型
pretrained_dir = f"{BASE_DIR}/examples/test_model/elmo"
i2v = get_pretrained_i2v("elmo_pp", model_dir=pretrained_dir)

SSLError: HTTPSConnectionPool(host='modelhub-backend-269-production.env.bdaa.pro', port=443): Max retries exceeded with url: /v1/api/getPretrainedModelList (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:997)')))

In [ ]:
i_vec, t_vec = i2v(test_items)
print(i_vec.shape)
print(t_vec.shape)
print()

# 也可以单独获取题目表征和各个token的表征
i_vec = i2v.infer_item_vector(test_items, key=lambda x: x['ques_content'])
print(i_vec.shape)
t_vec = i2v.infer_token_vector(test_items, key=lambda x: x['ques_content'])
print(t_vec.shape)
print()

# 同样，可以获取单个题目的表征
i_vec, t_vec = i2v(test_items[0], key=lambda x: x['ques_content'])
print(i_vec.shape)
print(t_vec.shape)